# Normal Image Pair Folder 만들기

> 단, Input <-> Normal  
Train,val <->  Train,val  
Test <-> Train,val  

--- 

> mimic_train(val, test) : /home/mskang/jinsu/med/H_LSTM_Transformer/data/new_data_mimic/mimic_train(val, test).txt


In [3]:
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
def load_list(file_list):
                                    # tag vector 들의 list
    filename_list = []                                  # image name 들의 list
    with open(file_list, 'r') as f:
        for line in f:
            items = line.split()
            image_name = items[0]                       # 해당 image name ex) CXR1972_IM-0633-1001
            image_name = '{}.jpg'.format(image_name)
            filename_list.append(image_name)
            
    return filename_list

In [5]:
train_txt = '/home/mskang/jinsu/med/H_LSTM_Transformer/data/new_data_mimic/mimic_train.txt'
train_filename_list=load_list(train_txt)
print(len(train_filename_list))
val_txt = '/home/mskang/jinsu/med/H_LSTM_Transformer/data/new_data_mimic/mimic_val.txt'
val_filename_list=load_list(val_txt)
print(len(val_filename_list))

865
108


In [6]:
train_filename_list = train_filename_list + val_filename_list
print(len(train_filename_list))

973


In [7]:
master='/home/mskang/jinsu/med/H_LSTM_Transformer/data/new_data_mimic/master_sheet.csv'

master=pd.read_csv(master)

# abnormal_idx=master['Normal']==0
normal_idx=master['Normal']==1

# abnormal_image_names = master['path'][abnormal_idx].str.slice(30, 74).values +'.jpg'
normal_image_names = master['path'][normal_idx].str.slice(30,74).values + '.jpg'
print('전체 normal image 개수 : ', len(normal_image_names))
# Only use Train / val
normal_image_names = [image  for image in normal_image_names if (image in train_filename_list)]
print('Train / VAl normal image 개수 :', len(normal_image_names))

input_image_names = master['path'].str.slice(30, 74).values + '.jpg'

전체 normal image 개수 :  360
Train / VAl normal image 개수 : 322


In [8]:
print(len(input_image_names), len(normal_image_names), len(master))

1083 322 1083


In [9]:
from image_similarity_measures.quality_metrics import rmse, ssim, sre

def compare_two_image(im1, im2, metric='rmse'):
    "im1,im2 : np.ndarray"
    
    if im1.shape != im2.shape:
        assert NameError('두 이미지의 크기가 같아야 합니다.')
    
    shape=im1.shape
    
    if len(shape)==2:
        im1=im1.reshape(shape+(1,))
        im2=im2.reshape(shape+(1,))
    
    if metric=='rmse':
        score=rmse(im1, im2)
    elif metric=='ssim':
        score=ssim(im1,im2)
    elif metric=='sre':
        score=sre(im1,im2)
    else:
        pass
    
    
    return score

In [10]:
image_root='/home/mskang/jinsu/med/H_LSTM_Transformer/data/all_jpgs'

In [11]:
from PIL import Image
import numpy as np
import pickle

In [15]:
pair_dict=pd.read_csv('pair_2.csv').iloc[:, 1:]

In [16]:
pair_dict = dict(pair_dict.values)

In [20]:
# pair_dict = {}


idx=840
for input_image_name in input_image_names[840:]:
    print('input_image_name : ', input_image_name)

    scores={}
    for normal_image_name in normal_image_names:
        
        normal_image =np.array(Image.open(os.path.join(image_root, normal_image_name)).resize((224,224)))
        input_image = np.array(Image.open(os.path.join(image_root, input_image_name)).resize((224,224)))
        scores[normal_image_name] = compare_two_image(normal_image, input_image)

    scores_sorted=sorted(scores.items(), key=lambda x:x[1])
    
    # scores : [['name1', 'best1_rmse'], ['name2', best2_rmse],..

    if scores_sorted[0][0] == input_image_name:
        pair_dict[input_image_name] = scores_sorted[1][0]
    else:
        pair_dict[input_image_name] = scores_sorted[0][0]
    print(f'complete,, {idx+1} / {len(input_image_names)}')
    print(f'best mage : {scores_sorted[1][0]}')
    print(f'best rmse : {scores_sorted[1][1]}')
    idx+=1
    

    if idx%5 == 0 :
        df = pd.DataFrame(list(pair_dict.items()), columns=['input_image', 'normal_image'])
        df.to_csv('pair_3.csv')






input_image_name :  70ff78b4-b7fd50a0-3be98615-4ba9b9d3-e5b89c48.jpg
complete,, 841 / 1083
best mage : 2bb86823-45f8d22b-81f8d920-9a1be431-aa5e4be0.jpg
best rmse : 0.009302952326834202
input_image_name :  cdb545cc-48c709fc-81d42016-8f067659-945277cf.jpg
complete,, 842 / 1083
best mage : e48a3b16-4f58b9f8-55d2e5a8-8656defd-989711ce.jpg
best rmse : 0.009710761718451977
input_image_name :  63304715-b1fad079-d2295ec2-4961f2d0-92d7ca4b.jpg
complete,, 843 / 1083
best mage : 17599eb3-173fc22b-84db3c0b-9639c478-d0db6bc2.jpg
best rmse : 0.009417667984962463
input_image_name :  9cd35b00-a86be493-766a825d-50389124-9f59c29b.jpg
complete,, 844 / 1083
best mage : 3533bc23-f6ad1649-f30c1c7d-01fdf5cd-b890884d.jpg
best rmse : 0.010037786327302456
input_image_name :  8dc876ae-427764ff-b4f508bf-26892673-6aa5e20a.jpg
complete,, 845 / 1083
best mage : fb863846-83697697-77131a5e-0b47a2cd-3ecda84f.jpg
best rmse : 0.012615076266229153
input_image_name :  e6c5bfd7-a5f92cbc-003b6164-2984d443-7ca2ebb8.jpg
comple

In [24]:
df = pd.DataFrame(list(pair_dict.items()), columns=['input_image', 'normal_image'])
df
df.to_csv('pair.csv')


In [91]:

print('input_image_name : ', _input)

input_image_name = _input
scores={}
for normal_image_name in normal_image_names:
    
    normal_image =np.array(Image.open(os.path.join(image_root, normal_image_name)).resize((224,224)))
    input_image = np.array(Image.open(os.path.join(image_root, input_image_name)).resize((224,224)))
    scores[normal_image_name] = compare_two_image(normal_image, input_image)

scores_sorted=sorted(scores.items(), key=lambda x:x[1])






input_image_name :  67cffbc1-8c5a9b31-d91c6a78-95a07a25-1f2e8c7a.jpg


1. Train_image_names, Test_image_names 나누어 놓기.
2. Train_image_names는 Train_image_names의 normal image와 비교(단, 자기 자신은 빼기)
3. Test_image_names는 Train_imgae_names의 normal image와만 비교


In [14]:
import pandas as pd
import shutil
import matplotlib.pyplot as plt


pair = pd.read_csv('pair.csv')

In [74]:
pair['normal_image'].value_counts()

36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg    22
6751667f-ec576bc3-8e7e44c5-00e2374d-64a3bdf7.jpg    18
6a662d77-487926b8-281db3bd-435c728a-23e3affa.jpg    16
e16a9d5c-17801488-28ba11c8-a5e364a8-89c4518d.jpg    16
47ea6acd-8b407807-eecf4cc6-6e25e9e6-c86a0966.jpg    14
                                                    ..
3ac68e75-77571934-ae24d154-f80e05b9-7ff3fd09.jpg     1
70ff78b4-b7fd50a0-3be98615-4ba9b9d3-e5b89c48.jpg     1
a39a9bc2-ab43dfcb-c0d7d7a5-2ecc806e-f2e4adb5.jpg     1
7fd82f55-aaba4935-f5b31bae-0288291b-82128657.jpg     1
076a5349-88bb9bda-2374918c-c4cf992b-2b6248f7.jpg     1
Name: normal_image, Length: 270, dtype: int64

In [77]:
pair[pair['normal_image']=='36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg']

,Unnamed: 0,input_image,normal_image
0,0,24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
6,6,748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
12,12,310ed1a6-ee306bc9-07d8d696-d5e3caa3-37749a62.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
30,30,05af2f67-a2b15808-de7dbcbb-2f89d568-7625fb4c.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
45,45,1a9b3b82-402d9e19-445679da-a0accb94-ebc21d66.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
71,71,4190d70f-dfa53c01-56ca25d3-eaa366ff-cffb3b6e.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
105,105,97d6f7dc-8ce699bc-9c5507fe-27e5bea5-ae40c5db.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
108,108,b66ea3a6-4a3de1b7-b29dea55-9654c9ea-28c9654e.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
133,133,f909ba00-164d827c-d0cf62bf-84d2bb17-a69abf2e.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
139,139,b966a2fd-63cd10f1-f98c4e5b-b897fa06-a1520bac.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg


In [15]:
pair

,Unnamed: 0,input_image,normal_image
0,0,24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4.jpg,36e73bcc-02a0174c-03d21a60-e442afe0-56a047d1.jpg
1,1,78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c.jpg,a06609d8-57f41c76-af85d92f-93c2b1de-217c4189.jpg
2,2,a770d8d6-7b6a62ff-815ab876-c81709a8-9a654a54.jpg,d940e7a9-559c248c-f6e05216-82a4fae8-7e84c087.jpg
3,3,8e457921-bc1af8aa-a65073c1-aaac8247-c5ceb780.jpg,21869e09-d73eea62-035063f1-2f271005-4eb55eba.jpg
4,4,62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a.jpg,0e923b0b-93746811-3ffa73b1-9baabf54-6b16de2a.jpg
...,...,...,...
1078,1078,78830bfd-5c6cded1-9c844764-6b3b5c2b-979da97b.jpg,02d7845f-0120d2f5-7dd7585d-ba0d6767-e6038823.jpg
1079,1079,a5c51c89-89622969-5c075934-3d5ec461-d6e25a19.jpg,3d38598e-6b5c6625-3d8099ff-21178a58-ea7859a1.jpg
1080,1080,9c01aecd-dc299044-a9fac53a-09a86bf1-58878e8a.jpg,076a5349-88bb9bda-2374918c-c4cf992b-2b6248f7.jpg
1081,1081,e88d062c-fed27e9f-bd163daf-d02bb4f0-d26dd112.jpg,0296bcb3-7e060a83-7da71b29-63ddb0a0-d1fc30b4.jpg


In [22]:
image_root='/home/mskang/jinsu/med/H_LSTM_Transformer/data/all_jpgs'
folder_path='/home/mskang/jinsu/med/XTransformer/mimic/normal_images/pair'

In [25]:
for i in range(len(pair)):
    copy_name = pair.iloc[i]['input_image']
    copy_file = pair.iloc[i]['normal_image']

    original_root = os.path.join(image_root, copy_file)
    new_root = os.path.join(folder_path, copy_name)

    shutil.copy(original_root, new_root)
 

### Check

In [29]:
import glob

In [67]:
new_images_path = glob.glob(os.path.join(folder_path, '*'))

new_images =[] 

for path in new_images_path :
    new_image = plt.imread(path)
    new_images.append(new_image)

In [40]:
import numpy as np

In [71]:

type(new_images[0]==new_images[7])

/home/mskang/.conda/envs/jsp_med/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


bool

In [72]:
count=0
for i in range(len(new_images)):
    for j in range(i+1, len(new_images)):

        mask = (new_images[i]==new_images[j])

        if type(mask) == bool:
            if mask ==True:
                count+=1
        if type(mask) ==np.ndarray:
            if (new_images[i]==new_images[j]).all():
                count+=1
        

/home/mskang/.conda/envs/jsp_med/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """


In [73]:
for i in range(len(new_images))

3145